# csv-pkl conversion
This notebooks provides util functions to transform `csv` files to `pkl` files and vice versa.

In [1]:
import os
import io
import ast
import re
import pandas as pd
import pickle

In [2]:
def create_dirs(path):
    dir_ = os.path.dirname(path)
    if not os.path.exists(dir_) or not os.path.isdir(dir_):
        os.makedirs(dir_)

## csv2pkl

In [3]:
def csv2pkl(src, dst=None, parse=True):
    if dst is None:
        dst = re.sub(r'\.csv$', '.pkl', src)
    
    # Loading
    df = pd.read_csv(src)
    
    # Parse
    if parse:
        for col in df.columns:
            try:
                df[col] = df[col].apply(lambda x: ast.literal_eval(x))
            except:
                pass
    
    # Overwrite?
    overwrite = True

    if os.path.exists(dst):
        while True:
            ans = str(input('A pkl file already exists. Overwrite? [y]/n: ')).lower()
            if ans == 'y' or ans == 'yes':
                overwrite = True
                break
            elif ans == 'n' or ans == 'no':
                overwrite = False
                break

    # Saving
    if overwrite:
        create_dirs(dst)
        data = df.to_dict('records')
        with open(dst, 'wb') as handle:
            pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)
        print(f'Saved at {dst}')

    return df

In [4]:
# example
# csv2pkl(src='../res/annotations/annotations_merged_balanced_renamed.csv', dst='../data/our_data/age_obj_cap_entries.pkl')

## pkl2csv

In [5]:
def pkl2csv(src, dst=None):
    if dst is None:
        dst = re.sub(r'\.pkl$', '.csv', src)
        
    # Loading
    with open(src, 'rb') as handle:
        obj = pickle.load(handle)
    df = pd.DataFrame(obj)
    
    # Overwrite?
    overwrite = True

    if os.path.exists(dst):
        while True:
            ans = str(input('A csv file already exists. Overwrite? [y]/n: ')).lower()
            if ans == 'y' or ans == 'yes':
                overwrite = True
                break
            elif ans == 'n' or ans == 'no':
                overwrite = False
                break

    # Saving
    if overwrite:
        create_dirs(dst)
        df.to_csv(dst)
        print(f'Saved at {dst}')
        
    return df

In [6]:
# example
# pkl2csv(src='../data/our_data/age_obj_cap_entries.pkl', dst='../data/our_data/age_obj_cap_entries.csv')

## Age dataset as pickle
In this section we make use of the previously defined functions to save age_dataset as a pickle file, to match the format required by the code.

In [7]:
csv2pkl(src='../data/age_data/age_dataset.csv')

A pkl file already exists. Overwrite? [y]/n: y
Saved at ../data/age_data/age_dataset.pkl


,Unnamed: 0,img_id,caption_list,pred_oscar,pred_sat,pred_nicplus,pred_niceq,bb_age
0,0,192,[A group of baseball players is crowded at the...,a baseball player holding a bat on top of a fi...,a batter catcher and umpire during a baseball ...,a baseball player holding a bat on top of a fi...,a baseball player holding a bat on a field.,old
1,1,241,[a man standing holding a game controller and ...,a man standing in a living room holding a nint...,a couple of people that are playing a video game,a group of people playing a game with nintendo...,a group of people playing a video game.,old
2,2,294,[A man standing in front of a microwave next t...,a man standing in front of a bunch of pots and...,a woman is pouring wine into a wine glass,a woman standing in a kitchen preparing food.,a man standing in a kitchen holding a knife.,old
3,3,328,[Three men in military suits are sitting on a ...,a group of three men sitting on top of a bench.,a group of people sitting on a bench,a black and white photo of a group of people s...,a black and white photo of a group of people s...,young
4,4,338,[Two people standing in a kitchen looking arou...,a couple of women standing in a kitchen next t...,a group of people standing in a kitchen,a woman standing in a kitchen next to a stove.,a woman standing in a kitchen next to a stove.,old
...,...,...,...,...,...,...,...,...
7033,7033,580613,[A woman is riding an orange sports motorcycle...,a woman riding on the back of a orange motorcy...,a woman riding a motorcycle on a city street,a woman riding on the back of a motorcycle.,a woman riding on the back of a motorcycle.,old
7034,7034,580847,[A young lady holding a tennis racquet in her ...,a woman holding a tennis racquet on a tennis c...,a woman holding a tennis racket on a tennis court,a woman holding a tennis racquet on a tennis c...,a woman holding a tennis racquet on a tennis c...,young
7035,7035,581040,[A young woman sits at her laptop near parked ...,a couple of women sitting next to each other a...,a woman is sitting in a car with a laptop,a woman sitting at a table with a laptop.,a woman sitting in a chair using a laptop comp...,old
7036,7036,581496,[A man holding the unpeeled end of a partially...,a man eating a donut while wearing a blue shirt.,a man holding a piece of food in his mouth,a man holding a cell phone up to his ear.,a man holding a cell phone up to his ear.,old
